In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
import json
from scipy.stats import norm
from scipy.interpolate import interp1d
import datetime
from tqdm import tqdm

In [2]:
# Directory containing the CSV files
loc=os.path.join('..','data')
# List maken voor df's
lst = []
# itereren
for filename in os.listdir(loc):
    if filename.endswith('.csv'):
        
        # extractie datum naam
        date_str = filename.split('.')[0]
        
        #filepath vastzetten
        file_path=os.path.join(loc, filename)
        
        #csv inlezen an df maken
        df=pd.read_csv(file_path)
        
        #datum kolom aan toevoegen
        df['datum']=date_str
        
        #kleine dataframes klaar per stuk, nu appenden in de lijst
        lst.append(df)
        
# lijst van df's concatineren to 1 DF
df_total=pd.concat(lst, ignore_index=True)

In [3]:
def clean_df(df):
    
    """
    Reinigt de DataFrame door outlier-datums te verwijderen, datumkolommen te converteren (datetimeobjecten),
    en dag- en maandnamen te categoriseren. Retourneert de gereinigde DataFrame alsook df_wrong.
    """
    
    # df_wrong met verkeerde datum (29 februari)
    outlier_datums = ['2019-2-29', '2019-1-1', '2019-7-1']
    df_wrong = df.loc[df['datum'].isin(outlier_datums)].copy()

    # Verwijder rijen met verkeerde datums
    df = df.loc[~df['datum'].isin(outlier_datums)].copy()
    df = df.reset_index(drop=True)
    
    # omzetten tot datetimes
    df['dag van het jaar'] = pd.to_datetime(df['datum']).dt.dayofyear
    df['verwachte datum'] = pd.to_datetime(df['verwachte datum'], format='%m/%d/%Y') # .dt.strftime('%Y-%m-%d')
    df['datum'] = pd.to_datetime(df['datum']) # .dt.strftime('%Y-%m-%d')

    # Define day and month categories
    day_categories = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    month_categories = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

    # Convert 'Day of Week' tot category: efficienter en zinvol (eg sorteringen dagen niet meer alfabetisch, maar volgens orde)
    df['weekdag'] = pd.to_datetime(df['datum']).dt.day_name()
    df['weekdag'] = pd.Categorical(df['weekdag'], categories=day_categories, ordered=True)

    # Add month name and categorize it
    df['maand'] = pd.to_datetime(df['datum']).dt.strftime('%B')
    df['maand'] = pd.Categorical(df['maand'], categories=month_categories, ordered=True)
    
    # Calculate the difference in days between 'verwachte datum' and 'datum'
    df['verschil_dagen'] = (df['verwachte datum'] - df['datum']).dt.days

    return df, df_wrong

In [4]:
df_clean, df_wrong=clean_df(df_total)

In [5]:
df_clean

,gemeente,naam,geslacht,verwachte datum,datum,dag van het jaar,weekdag,maand,verschil_dagen
0,Merksplas,Liliane,Vrouwelijk,2019-01-04,2019-01-10,10,Thursday,January,-6
1,Bornem,Jeroen,Mannelijk,2019-01-10,2019-01-10,10,Thursday,January,0
2,Houthalen-Helchteren,Fatima,Vrouwelijk,2019-01-21,2019-01-10,10,Thursday,January,11
3,Lievegem,Hilaire,Mannelijk,2019-01-15,2019-01-10,10,Thursday,January,5
4,Spiere-Helkijn,Christian,Mannelijk,2019-01-16,2019-01-10,10,Thursday,January,6
...,...,...,...,...,...,...,...,...,...
115388,Lede,Lobke,Vrouwelijk,2019-09-12,2019-09-09,252,Monday,September,3
115389,Westerlo,Willem,Mannelijk,2019-09-04,2019-09-09,252,Monday,September,-5
115390,Brugge,Diane,Vrouwelijk,2019-09-20,2019-09-09,252,Monday,September,11
115391,Lendelede,Marie-Rose,Vrouwelijk,2019-10-28,2019-09-09,252,Monday,September,49


In [6]:
df_wrong

,gemeente,naam,geslacht,verwachte datum,datum
0,Hooglede,Elias,Mannelijk,01/14/2019,2019-1-1
1,Sint-Niklaas (Sint-Niklaas),Pauline,Vrouwelijk,01/05/2019,2019-1-1
2,Wijnegem,Anita,Vrouwelijk,01/05/2019,2019-1-1
3,Grâce-Hollogne,Jean-Paul,Mannelijk,01/13/2019,2019-1-1
4,Boussu,François,Mannelijk,01/26/2019,2019-1-1
...,...,...,...,...,...
87731,Oostende,Magali,Vrouwelijk,07/15/2019,2019-7-1
87732,Rotselaar,Jean,Mannelijk,06/30/2019,2019-7-1
87733,Brussel,Lina,Vrouwelijk,07/11/2019,2019-7-1
87734,Hamont-Achel,Brenda,Vrouwelijk,06/25/2019,2019-7-1


In [7]:
df_clean.dtypes

gemeente                    object
naam                        object
geslacht                    object
verwachte datum     datetime64[ns]
datum               datetime64[ns]
dag van het jaar             int32
weekdag                   category
maand                     category
verschil_dagen               int64
dtype: object

In [8]:
df_wrong.dtypes

gemeente           object
naam               object
geslacht           object
verwachte datum    object
datum              object
dtype: object

In [9]:
df_wrong.sample(30)

,gemeente,naam,geslacht,verwachte datum,datum
87459,Sint-Gillis-Waas,Carine,Vrouwelijk,07/15/2019,2019-7-1
87669,Evergem,Pierre,Mannelijk,07/15/2019,2019-7-1
46,Antwerpen,Yamina,Vrouwelijk,01/07/2019,2019-1-1
229,Wezet,Julien,Mannelijk,01/10/2019,2019-1-1
87234,Genk,Fabio,Mannelijk,07/09/2019,2019-7-1
86827,Saint-Ghislain,Catherine,Vrouwelijk,07/07/2019,2019-7-1
399,Eigenbrakel,Christian,Mannelijk,01/18/2019,2019-1-1
87641,Westerlo,Maria,Vrouwelijk,07/06/2019,2019-7-1
211,Hannuit,Thomas,Mannelijk,12/31/2018,2019-1-1
322,Eigenbrakel,Maximilien,Mannelijk,01/05/2019,2019-1-1
